^fill^README.md^here^

^fill^theory.md^here^

## Demonstration

### 1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

In [ ]:
# Standard library imports
from pathlib import Path
import os
import sys
import uuid
import shutil
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np 

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

print('Notebook last executed on', datetime.date.today(), 'using iprPy version', iprPy.__version__)

#### 1.2. Default calculation setup

In [ ]:
# Specify calculation style
calc_style = 'stacking_fault_static'

# If workingdir is already set, then do nothing (already in correct folder)
try:
    workingdir = workingdir

# Change to workingdir if not already there
except:
    workingdir = Path('calculationfiles', calc_style)
    if not workingdir.is_dir():
        workingdir.mkdir(parents=True)
    os.chdir(workingdir)

### 2. Assign values for the calculation's run parameters

#### 2.1 Specify system-specific paths

- __lammps_command__ is the LAMMPS command to use.

- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

In [ ]:
lammps_command = 'lmp_serial'
mpi_command = None

#### 2.2. Load interatomic potential

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for the calculation.  

- __potential_file__ gives the path to the potential_LAMMPS reference record to use.  Here, this parameter is automatically generated using potential_name and librarydir.

- __potential_dir__ gives the path for the folder containing the artifacts associated with the potential (i.e. eam.alloy file).  Here, this parameter is automatically generated using potential_name and librarydir.

- __potential__ is an atomman.lammps.Potential object (required).  Here, this parameter is automatically generated from potential_file and potential_dir.

In [ ]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'

# Define potential_file and potential_dir using librarydir and potential_name
potential_file = Path(iprPy.libdir, 'potential_LAMMPS', f'{potential_name}.json')
potential_dir = Path(iprPy.libdir, 'potential_LAMMPS', potential_name)

# Initialize Potential object using potential_file and potential_dir.
potential = lmp.Potential(potential_file, potential_dir)
print('Successfully loaded potential', potential)

#### 2.3. Load initial unit cell system

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to load. 

- __symbols__ is a list of the potential's elemental model symbols to associate with the unique atom types of the loaded system. 

- __box_parameters__ is a list of the a, b, c lattice constants to assign to the loaded file.

- __load_file__ gives the path to the atomic configuration file to load for the ucell system.  Here, this is generated automatically using prototype_name and librarydir.

- __load_style__ specifies the format of load_file.  Here, this is automatically set for crystal_prototype records.

- __load_options__ specifies any other keyword options for properly loading the load_file.  Here, this is automatically set for crystal_prototype records.

- __ucell__ is an atomman.System representing a fundamental unit cell of the system (required).  Here, this is generated using the load parameters and symbols.

In [ ]:
prototype_name = 'A1--Cu--fcc'
symbols = ['Ni']
box_parameters = uc.set_in_units([3.52, 3.52, 3.52], 'angstrom')

# Define load_file using librarydir and prototype_name
load_file = Path(iprPy.libdir, 'crystal_prototype', f'{prototype_name}.json')

# Define load_style and load_options for crystal_prototype records
load_style = 'system_model'
load_options = {}

# Create ucell by loading prototype record
ucell = am.load(load_style, load_file, symbols=symbols, **load_options)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], scale=True)

print(ucell)

#### 2.4. Specify the defect parameters

- __stackingfault_name__ gives the name of the stacking_fault reference record in the iprPy library to use. 

- __shiftfraction1__ is the fraction of shiftvector1 to apply in generating the fault.

- __shiftfraction2__ is the fraction of shiftvector2 to apply in generating the fault.

- __stackingfault_file__ gives the path to the stacking_fault reference record to use. Here, this is built automatically using stackingfault_name and librarydir.

- __cutboxvector__ specifies which of the three box vectors ('a', 'b', or 'c') is to be made non-periodic to create the free surface.  Here, this is extracted from the defect model.

- __faultpos__ specifies the relative position within the rotated cell before applying sizemults where the fault plane is placed.  Here, this is extracted from the defect model.

- __shiftvector1, shiftvector2__ specify two non-parallel Miller crystal vectors within the fault plane corresponding to full planar shifts from one perfect crystal configuration to another.  Here, these are extracted from the defect model.

In [ ]:
stackingfault_name = 'A1--Cu--fcc--111sf'

shiftfraction1 = 0.5
shiftfraction2 = 0.0

# Define surface_file using librarydir and surface_name
stackingfault_file = Path(iprPy.libdir, 'stacking_fault', f'{stackingfault_name}.json')

# Parse freesurface_file using iprPy.input.interpret()
defectinputs = {'stackingfault_file':stackingfault_file}
iprPy.input.subset('stackingfault').interpret(defectinputs)

# Extract parameters from defect model
cutboxvector = defectinputs['stackingfault_cutboxvector']
faultpos = defectinputs['stackingfault_faultpos']
shiftvector1 = defectinputs['stackingfault_shiftvector1']
shiftvector2 = defectinputs['stackingfault_shiftvector2']
print('cutboxvector =', cutboxvector)
print('faultpos =', faultpos)
print('shiftvector1 =', shiftvector1)
print('shiftvector2 =', shiftvector2)

#### 2.5. Modify system

- __a_uvw__ Miller crystal vector used to generate a rotated system from ucell.  The a box vector of the rotated system will correspond to the crystal vector relative to ucell.  Here, this is extracted from the defect model.

- __b_uvw__ Miller crystal vector used to generate a rotated system from ucell.  The a box vector of the rotated system will correspond to the crystal vector relative to ucell.  Here, this is extracted from the defect model.

- __c_uvw__ Miller crystal vector used to generate a rotated system from ucell.  The a box vector of the rotated system will correspond to the crystal vector relative to ucell.  Here, this is extracted from the defect model.

- __atomshift__ fractional rigid body shift to apply to the rotated system. Here, this is extracted from the defect model.

- __sizemults__ list of three integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in creating system.

- __system__ is an atomman.System to use as the input system for the calculation (required). 

In [ ]:
sizemults = [5, 5, 10]

# Extract parameters from defect model
a_uvw = np.array(defectinputs['a_uvw'].strip().split(), dtype=float)
b_uvw = np.array(defectinputs['b_uvw'].strip().split(), dtype=float)
c_uvw = np.array(defectinputs['c_uvw'].strip().split(), dtype=float)
atomshift = np.array(defectinputs['atomshift'].strip().split(), dtype=float)
print('a_uvw =', a_uvw)
print('b_uvw =', b_uvw)
print('c_uvw =', c_uvw)
print('atomshift =', atomshift)

# Rotate to specified uvws
system, transform = ucell.rotate(np.array([a_uvw, b_uvw, c_uvw]), return_transform=True)

# Scale atomshift by rotated system's box vectors
shift = (atomshift[0] * system.box.avect +
         atomshift[1] * system.box.bvect +
         atomshift[2] * system.box.cvect)
system.atoms.pos += shift

# Generate system by supersizing ucell
system = system.supersize(*sizemults)
print('# of atoms in system =', system.natoms)

#### 2.6. Modify defect parameters based on modified system information

- __shiftvector1, shiftvector2__ are converted from strings to arrays

- __faultpos__ is adjusted to be relative to the supercell system by placing the cut inside the center cell replica.

In [ ]:
# Convert shift vectors from strings to arrays
shiftvector1 = np.array(shiftvector1.strip().split(), dtype=float)
shiftvector2 = np.array(shiftvector2.strip().split(), dtype=float)

# Identify number of size multiples, m, along cutboxvector
if   cutboxvector == 'a': 
    m = sizemults[0]
elif cutboxvector == 'b': 
    m = sizemults[1]
elif cutboxvector == 'c': 
    m = sizemults[2]

# For odd m, initial position of 0.5 goes to 0.5
if m % 2 == 1:
    faultpos = (faultpos + (m-1) * 0.5) / m
# For even m, initial position of 0.0 goes to 0.5
else:
    faultpos = (2 * faultpos + m) / (2 * m)

print('faultpos =', faultpos)
print('shiftvector1 =', shiftvector1)
print('shiftvector2 =', shiftvector2)

#### 2.6. Specify calculation-specific run parameters

- __energytolerance__ is the energy tolerance to use during the minimizations. This is unitless.

- __forcetolerance__ is the force tolerance to use during the minimizations. This is in energy/length units.

- __maxiterations__ is the maximum number of minimization iterations to use.

- __maxevaluations__ is the maximum number of minimization evaluations to use.

- __maxatommotion__ is the largest distance that an atom is allowed to move during a minimization iteration. This is in length units.

In [ ]:
energytolerance = 1e-8
forcetolerance = uc.set_in_units(0.0, 'eV/angstrom')
maxiterations = 10000
maxevaluations = 100000
maxatommotion = uc.set_in_units(0.01, 'angstrom')

### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1 sfmin.template

In [ ]:
with open('sfmin.template', 'w') as f:
    f.write("""^fill^sfmin.template^here^""")

#### 3.2 stackingfaultrelax()

In [ ]:
^fill^calc_stacking_fault_static.py(stackingfaultrelax)^here^

#### 3.3 stackingfault()

In [ ]:
^fill^calc_stacking_fault_static.py(stackingfault)^here^

### 4. Run calculation function(s)

In [ ]:
results_dict = stackingfault(lammps_command, system, potential,
                             mpi_command = mpi_command,
                             a1vect = shiftvector1,
                             a2vect = shiftvector2,
                             ucell = ucell,
                             transform = transform,
                             cutboxvector = cutboxvector,
                             faultposrel = faultpos,
                             a1 = shiftfraction1,
                             a2 = shiftfraction2,
                             etol = energytolerance,
                             ftol = forcetolerance,
                             maxiter = maxiterations,
                             maxeval = maxevaluations,
                             dmax = maxatommotion)

In [ ]:
results_dict.keys()

### 5. Report results

#### 5.1 Define units for outputting values

- __length_unit__ is the unit of area to display delta displacemets in.

- __area_unit__ is the unit of area to display fault area in.

- __energyperarea_unit__ is the energy per area to report the surface energy in.

In [ ]:
length_unit = 'nm'
area_unit = 'nm^2'
energyperarea_unit = 'mJ/m^2'

#### 5.2 Print $A_{fault}$, $E_{gsf}$, and $\Delta\delta$

In [ ]:
print('Values for fractional shift = (%f, %f)' % (shiftfraction1, shiftfraction2))
print('A_fault =    ', uc.get_in_units(results_dict['A_fault'], area_unit), area_unit)
print('E_gsf =      ', uc.get_in_units(results_dict['E_gsf'], energyperarea_unit), energyperarea_unit)
print('delta_disp = ', uc.get_in_units(results_dict['delta_disp'], length_unit), length_unit)